In [234]:
import pandas as pd
from great_expectations.data_context import FileDataContext
import great_expectations as gx
import json

# Data Load to Check

In [235]:
# Load the cleaned CSV file
path_to_data = "C:\\Users\\Affan\\Hacktiv8\\Phase 2\\Milestone\\p2-ftds024-hck-m3-Baldaric\\dags\\P2M3_Affan_data_clean.csv"
df = pd.read_csv(path_to_data)

# Display the first few rows
df.head()

,result_id,round,race_name,circuit_name,date,constructor_name,constructor_points,constructor_rank,driver_name,driver_code,driver_points,driver_rank,grid,position,fastest_lap,laps,race_points
0,1,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,McLaren,14.0,1,Lewis Hamilton,HAM,10.0,1,1,1.0,39.0,58,10.0
1,2,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,BMW Sauber,8.0,3,Nick Heidfeld,HEI,8.0,2,5,2.0,41.0,58,8.0
2,3,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,Williams,9.0,2,Nico Rosberg,ROS,6.0,3,7,3.0,41.0,58,6.0
3,4,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,Renault,5.0,4,Fernando Alonso,ALO,5.0,4,11,4.0,58.0,58,5.0
4,5,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,McLaren,14.0,1,Heikki Kovalainen,KOV,4.0,5,3,5.0,43.0,58,4.0


In [236]:
print(df.columns.tolist())

['result_id', 'round', 'race_name', 'circuit_name', 'date', 'constructor_name', 'constructor_points', 'constructor_rank', 'driver_name', 'driver_code', 'driver_points', 'driver_rank', 'grid', 'position', 'fastest_lap', 'laps', 'race_points']


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6973 entries, 0 to 6972
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   result_id           6973 non-null   int64  
 1   round               6973 non-null   int64  
 2   race_name           6973 non-null   object 
 3   circuit_name        6973 non-null   object 
 4   date                6973 non-null   object 
 5   constructor_name    6973 non-null   object 
 6   constructor_points  6973 non-null   float64
 7   constructor_rank    6973 non-null   int64  
 8   driver_name         6973 non-null   object 
 9   driver_code         6973 non-null   object 
 10  driver_points       6973 non-null   float64
 11  driver_rank         6973 non-null   int64  
 12  grid                6973 non-null   int64  
 13  position            6973 non-null   float64
 14  fastest_lap         6973 non-null   float64
 15  laps                6973 non-null   int64  
 16  race_p

# Great Expectations

In [238]:
context = FileDataContext.create(project_root_dir='./')

In [239]:
# Get the GE context
context = gx.get_context()

# Define the datasource name
datasource_name = "csv-data-formula1"

# Check if the datasource already exists and remove it
if datasource_name in context.datasources:
    context.delete_datasource(datasource_name)
    print(f"Deleted existing datasource: {datasource_name}")

# Now safely add the new datasource
datasource = context.sources.add_pandas(datasource_name)

# Define asset name and path
asset_name = "formula1"
path_to_data = "C:\\Users\\Affan\\Hacktiv8\\Phase 2\\Milestone\\p2-ftds024-hck-m3-Baldaric\\dags\\P2M3_Affan_data_clean.csv"

# Add CSV asset
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()


Deleted existing datasource: csv-data-formula1


In [240]:
# Create an expectation suite
expectation_suite_name = 'expectation-f1-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,result_id,round,race_name,circuit_name,date,constructor_name,constructor_points,constructor_rank,driver_name,driver_code,driver_points,driver_rank,grid,position,fastest_lap,laps,race_points
0,1,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,McLaren,14.0,1,Lewis Hamilton,HAM,10.0,1,1,1.0,39.0,58,10.0
1,2,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,BMW Sauber,8.0,3,Nick Heidfeld,HEI,8.0,2,5,2.0,41.0,58,8.0
2,3,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,Williams,9.0,2,Nico Rosberg,ROS,6.0,3,7,3.0,41.0,58,6.0
3,4,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,Renault,5.0,4,Fernando Alonso,ALO,5.0,4,11,4.0,58.0,58,5.0
4,5,1,Australian Grand Prix,Albert Park Grand Prix Circuit,2008-03-16,McLaren,14.0,1,Heikki Kovalainen,KOV,4.0,5,3,5.0,43.0,58,4.0


## Expectations

### 1. To Be Unique

In [241]:
validator.expect_column_values_to_be_unique(column='result_id')

# Save the expectation
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

### 2. To Be Min-Max Values

In [242]:
# Define numerical columns with min & max constraints
MinMax_limit = {
    "constructor_points": (0, 1000),
    "constructor_rank": (1, 12),
    "race_points": (0, 50)
}

# Loop to apply the range
for i, (MinVal, MaxVal) in MinMax_limit.items():
    validator.expect_column_values_to_be_between(column=i, min_value=MinVal, max_value=MaxVal)

# Save expectations
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

### 3. Race Point To Be in set

In [243]:
ValDriverCode = df["driver_code"].unique().tolist()
validator.expect_column_values_to_be_in_set(
    column="driver_code",
    value_set=ValDriverCode
)

validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

### 4. Columns to Have Specific Data Types

In [244]:
# Define column data types
DataTypes = {
    "round": 'int64',
    "race_name": 'str',
    "circuit_name": 'str',
    "date": 'str',
    "constructor_name": 'str',
    "constructor_points": 'float',
    "constructor_rank": 'int64',
    "driver_name": 'str',
    "driver_code": 'str',
    "driver_points": 'float',
    "driver_rank": 'int64',
    "grid": 'int64',
    "fastest_lap": 'float',
    "laps": 'int64',
    "race_points": 'float'
}

# Apply data type expectation
for i, CorrectType in DataTypes.items():
    validator.expect_column_values_to_be_of_type(column=i, type_=CorrectType)

# Save expectations
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

### 5. Match DateTime

In [245]:
# Apply expectation for date format
validator.expect_column_values_to_match_strftime_format(
    column="date",
    strftime_format="%Y-%m-%d"
)

# Save expectations
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

### 6. Column Not Be Null

In [246]:
# Column to check
NoNull = ["circuit_name", "date", "driver_code"]

# Loop to apply expectation
for i in NoNull:
    validator.expect_column_values_to_not_be_null(column=i)

# Save the expectation
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

### 7. Column Must Exist

In [247]:
# Column to check
Existence = ["result_id", "constructor_name", "driver_name", "date", "grid", "position"]


# Loop to apply expectation
for i in Existence:
    validator.expect_column_to_exist(column=i)

# Save the expectation
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

## Expectation Check

In [248]:
validator.validate()

Calculating Metrics:   0%|          | 0/62 [00:00<?, ?it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_unique",
        "kwargs": {
          "column": "result_id",
          "batch_id": "csv-data-formula1-formula1"
        },
        "meta": {}
      },
      "result": {
        "element_count": 6973,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_to_exist",
        "kwargs": {
          "column": "result_id",
          "batch_id": "csv-data-formula1-formula1"
      

In [249]:
Result = validator.validate()
ResultDict = Result.to_json_dict()
ResultDict

Calculating Metrics:   0%|          | 0/62 [00:00<?, ?it/s]

{'success': True,
 'results': [{'success': True,
   'expectation_config': {'expectation_type': 'expect_column_values_to_be_unique',
    'kwargs': {'column': 'result_id',
     'batch_id': 'csv-data-formula1-formula1'},
    'meta': {}},
   'result': {'element_count': 6973,
    'unexpected_count': 0,
    'unexpected_percent': 0.0,
    'partial_unexpected_list': [],
    'missing_count': 0,
    'missing_percent': 0.0,
    'unexpected_percent_total': 0.0,
    'unexpected_percent_nonmissing': 0.0},
   'meta': {},
   'exception_info': {'raised_exception': False,
    'exception_traceback': None,
    'exception_message': None}},
  {'success': True,
   'expectation_config': {'expectation_type': 'expect_column_to_exist',
    'kwargs': {'column': 'result_id',
     'batch_id': 'csv-data-formula1-formula1'},
    'meta': {}},
   'result': {},
   'meta': {},
   'exception_info': {'raised_exception': False,
    'exception_traceback': None,
    'exception_message': None}},
  {'success': True,
   'expecta

In [250]:
AllCheck = len(ResultDict['results'])
SuccessCheck = 0
FailCheck = 0

for i in ResultDict['results']:
    if i['success']:
         SuccessCheck += 1
    else:
        FailCheck += 1

print(f'Data to be checked: {AllCheck}')
print(f'Data in accordance with expectations: {SuccessCheck}')
print(f'Data not in accordance with expectations: {FailCheck}')

Data to be checked: 30
Data in accordance with expectations: 30
Data not in accordance with expectations: 0
